In [ ]:
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf 
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip drive/MyDrive/93Gait_1Session.zip


Archive:  drive/MyDrive/93Gait_1Session.zip
   creating: 93Gait_1Session/
   creating: 93Gait_1Session/2Files/
  inflating: 93Gait_1Session/2Files/sub1-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub1-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub10-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub10-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub11-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub11-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub13-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub13-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub14-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub14-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub15-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub15-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub16-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub16-rp-s1.csv  
  inflating: 93Gait_1Session/2Files/sub17-lw-s1.csv  
  inflating: 93Gait_1Session/2Files/sub17-rp-s1.csv.csv  
  inflating: 93Gait_1Se

In [ ]:
import re

# create a list of file paths for all CSV files in the specified directory
csv_files = glob.glob('93Gait_1Session/2Files/*.csv')

# create dictionaries to store the left waist and right pocket dataframes for each subject
left_dfs = {}
right_dfs = {}
merged_dfs = {}
# define regular expressions to extract subject ID and sensor location from the file names
subject_regex = re.compile(r'sub(\d+)-')
location_regex = re.compile(r'-(lw|rp)-')

# loop over each CSV file
for file in csv_files:
    # extract the subject ID and sensor location from the file name
    subject_id = 'sub' + subject_regex.search(file).group(1)
    location = location_regex.search(file).group(1)
    
    # read in the CSV file as a dataframe
    df = pd.read_csv(file)
    
    # convert timestamps to datetime objects
    df['accelerometerTimestamp_sinceReboot.s.'] = pd.to_datetime(df['accelerometerTimestamp_sinceReboot.s.'], unit='s')
    df['gyroTimestamp_sinceReboot.s.'] = pd.to_datetime(df['gyroTimestamp_sinceReboot.s.'], unit='s')
    
    # extract desired features into a new DataFrame
    new_df = df[['accelerometerTimestamp_sinceReboot.s.', 'motionUserAccelerationX.G.', 'motionUserAccelerationY.G.', 'motionUserAccelerationZ.G.', 'gyroTimestamp_sinceReboot.s.', 'gyroRotationX.rad.s.', 'gyroRotationY.rad.s.', 'gyroRotationZ.rad.s.']]
    
    # reset index of the new DataFrame to ensure that the index values are unique
    new_df.reset_index(drop=True, inplace=True)

    # add the new DataFrame to the appropriate dictionary based on the sensor location
    if location == 'lw':
        if subject_id not in left_dfs:
            left_dfs[subject_id] = new_df
        else:
            left_dfs[subject_id] = pd.concat([left_dfs[subject_id], new_df])
    elif location == 'rp':
        if subject_id not in right_dfs:
            right_dfs[subject_id] = new_df
        else:
            right_dfs[subject_id] = pd.concat([right_dfs[subject_id], new_df])
    for subject in left_dfs:
      if subject in right_dfs:
        left_dfs[subject].reset_index(drop=True, inplace=True)
        right_dfs[subject].reset_index(drop=True, inplace=True)
        merged_dfs[subject] = pd.concat([left_dfs[subject], right_dfs[subject]], axis=1)
 




In [ ]:
len(merged_dfs)

62

In [ ]:
merged_dfs['sub86']

,accelerometerTimestamp_sinceReboot.s.,motionUserAccelerationX.G.,motionUserAccelerationY.G.,motionUserAccelerationZ.G.,gyroTimestamp_sinceReboot.s.,gyroRotationX.rad.s.,gyroRotationY.rad.s.,gyroRotationZ.rad.s.,accelerometerTimestamp_sinceReboot.s.,motionUserAccelerationX.G.,motionUserAccelerationY.G.,motionUserAccelerationZ.G.,gyroTimestamp_sinceReboot.s.,gyroRotationX.rad.s.,gyroRotationY.rad.s.,gyroRotationZ.rad.s.
0,1970-01-01 02:04:08.240620625,0.052723,-0.228374,-0.196036,1970-01-01 02:04:08.238666625,0.304952,-0.094589,0.281103,1970-01-02 02:12:13.171510125,-0.162292,0.246343,0.133538,1970-01-02 02:12:13.161409125,1.947771,-0.478249,0.077102
1,1970-01-01 02:04:08.250599625,0.019121,-0.217521,-0.195304,1970-01-01 02:04:08.248646625,0.334012,-0.067051,0.330720,1970-01-02 02:12:13.181490125,-0.110990,0.319944,0.052234,1970-01-02 02:12:13.171419125,2.023672,-0.557080,0.002613
2,1970-01-01 02:04:08.260578625,-0.006514,-0.179245,-0.150029,1970-01-01 02:04:08.258625625,0.342130,-0.059696,0.356448,1970-01-02 02:12:13.191499125,-0.007672,0.360528,0.029169,1970-01-02 02:12:13.181428125,1.958378,-0.563697,-0.210267
3,1970-01-01 02:04:08.270557625,-0.026829,-0.123116,-0.109844,1970-01-01 02:04:08.268604625,0.315481,-0.058671,0.354958,1970-01-02 02:12:13.201479125,0.045457,0.312493,0.068573,1970-01-02 02:12:13.191438125,1.892260,-0.540777,-0.386000
4,1970-01-01 02:04:08.280567625,-0.034402,-0.065148,-0.114704,1970-01-01 02:04:08.278614625,0.308007,-0.048023,0.354906,1970-01-02 02:12:13.211488125,0.045332,0.225307,0.136859,1970-01-02 02:12:13.201417125,1.890213,-0.495151,-0.539879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46995,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1970-01-02 02:20:32.842120375,-0.006724,0.536192,0.324892,1970-01-02 02:20:32.833270375,-0.265439,0.826844,0.377459
46996,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1970-01-02 02:20:32.852099375,-0.004074,0.514277,0.333384,1970-01-02 02:20:32.843280375,-0.112310,0.713299,0.310362
46997,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1970-01-02 02:20:32.862109375,-0.033805,0.489447,0.329860,1970-01-02 02:20:32.853290375,0.100085,0.571654,0.237637
46998,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,1970-01-02 02:20:32.872088375,-0.060187,0.475397,0.324516,1970-01-02 02:20:32.863299375,0.408126,0.413370,0.146788


In [ ]:
for subject in merged_dfs:
  merged_dfs[subject].drop(['accelerometerTimestamp_sinceReboot.s.','gyroTimestamp_sinceReboot.s.'],axis=1, inplace=True)

In [ ]:
merged_dfs['sub93'].shape

(49000, 12)

In [ ]:
merged_dfs['sub86'].isnull().sum()

motionUserAccelerationX.G.    1000
motionUserAccelerationY.G.    1000
motionUserAccelerationZ.G.    1000
gyroRotationX.rad.s.          1000
gyroRotationY.rad.s.          1000
gyroRotationZ.rad.s.          1000
motionUserAccelerationX.G.       0
motionUserAccelerationY.G.       0
motionUserAccelerationZ.G.       0
gyroRotationX.rad.s.             0
gyroRotationY.rad.s.             0
gyroRotationZ.rad.s.             0
dtype: int64

In [ ]:
for key, df in merged_dfs.items():
    if df.isnull().values.any():
        merged_dfs[key] = df.interpolate(method='pad', limit_direction='forward')


In [ ]:

window_size=128
overlap = 50
num_features = 12


X_data=[]
y_data=[]
for subject, df in merged_dfs.items():
    # normalize data using MinMaxScaler
    scaler = MinMaxScaler()
    df = scaler.fit_transform(df)   
    
    # create a TimeSeriesGenerator object with the specified window size and overlap
    generator = TimeseriesGenerator(df, [0]*len(df), length=window_size, batch_size=32)
    
    # loop over each window of data generated by the TimeSeriesGenerator
    for i in range(len(generator)):
        # extract the X and y data for the current window
        X, y = generator[i]
        
        # append the X data to the list of all X data
        X_data.append(X)
        
        # append the subject label to the list of all y data
        y_data.append(subject)

In [ ]:

# assuming your data is stored in merged_dfs dictionary

X_train_data = []
y_train_data = []

X_test_data = []
y_test_data = []


window_size = 128
overlap = 50
num_features = 12

# create a list of labels (y) from the keys of merged_dfs dictionary
#labels = list(Input_dfs.keys())
# concatenate all labels into one dataframe


#all_labels = pd.concat([pd.Series(np.full(len(df), label), name='label').reset_index(drop=True) for label, df in merged_dfs.items()], axis=0)
all_labels = pd.concat([pd.Series(np.full(len(df), label), name='label') for label, df in merged_dfs.items()], axis=0, ignore_index=True)


print("All labels shape:", all_labels.shape)
print("Keys in merged_dfs:", all_labels.head(130))


# concatenate all dataframes into one dataframe
all_data = pd.concat([df for df in merged_dfs.values()], axis=0)

# normalize the data using MinMaxScaler
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(all_data)

# encode the labels using LabelEncoder
label_encoder = LabelEncoder()
#all_labels_encoded = label_encoder.fit_transform(all_labels)
all_labels_encoded = label_encoder.fit_transform(all_labels) + 1 

# split the normalized data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(normalized_data, all_labels_encoded, test_size=0.2, shuffle=False)
print(len(X_test))
print(len(y_test))

# create a TimeSeriesGenerator object for training data
train_generator = TimeseriesGenerator(X_train, y_train, length=window_size, sampling_rate=1,batch_size=32)
print(len(train_generator))

# create a TimeSeriesGenerator object for testing data
test_generator = TimeseriesGenerator(X_test, y_test, length=window_size,sampling_rate=1, batch_size=32)
print(len(test_generator))

In [ ]:
model = tf.keras.Sequential()

model.add(tf.keras.layers. LSTM(128, input_shape= (window_size, num_features), return_sequences=True)) 
model.add(tf.keras.layers.LeakyReLU(alpha=0.5))

model.add(tf.keras.layers. LSTM(128, return_sequences=True))

model.add(tf.keras.layers. LeakyReLU (alpha=0.5))
model.add(tf.keras.layers.Dropout (0.3))

model.add(tf.keras. layers. LSTM(64, return_sequences=False))

model.add(tf.keras.layers.Dropout (0.3)) 
#model.add(tf.keras.layers.Dense (1))
#model.add(Dense(len(label_encoder.classes_), activation='softmax'))
model.add(Dense((62), activation='softmax'))